# Projet-optimisation

## Partie 1

1)
L'expression (2) correspond aux bénéfices de la boulangerie : on soustraît les entrées d'argent par les dépenses. En particulier, on utilise min{q,d} pour obtenir la quantité vendue de chaque produit (qu'on multiplie par le prix de vente pour obtenir le revenu). En effet, q étant la quantité fabriquée et d la demande, la quantité réellement vendue correspond au minimum des deux : impossible de vendre plus que ce qu'on a produit, ni plus que ce que les clients voudront acheter.

2)
Ce dernier terme n'est cependant pas différentiable, ce qui rend inutilisable la majorité des algorithmes d'optimisation que nous avons étudiées en cours.

3)
Soit $i \in \llbracket 1,p \rrbracket$ Supposons que $q_i>d_i$ On a :
$$
h_i(q,d) = \frac{q_ie^{-\alpha q_i} + d_ie^{-\alpha d_i}}{e^{-\alpha q_i} + e^{-\alpha d_i}}
$$
$$
h_i(q,d) = \frac{q_ie^{\alpha (d_i - q_i)} + d_i}{e^{\alpha (d_i - q_i)} + 1}
$$
Or, $d_i - q_i < 0$, donc pour $\alpha \longrightarrow +\infty$ :
$$
h_i(q,d) \longrightarrow d_i
$$
le raisonnement pour $q_i$ est symétrique.

On a donc finalement :
$$
h(q,d)\underset{\alpha\to+\infty}{\longrightarrow}min(q,d)
$$

L'intérêt de s'intéresser à ce problème plutôt qu'au précédent est que la fonction h est différentiable. Nous pourrons donc utiliser des algorithmes d'optimisations qui ne s'appliquent qu'à des fonctions différentiables.

4)
On veut maximiser $v^T h(q,d) - c^T r$, c'est-à-dire minimiser $-v^T h(q,d) + c^T r$.

La contrainte qui s'applique à la production est qu'on ne peut produire sans matières premières, ce qui se réécrit $r >= Aq$

On veut également une quantité fabriquée positive, soit $q > 0$

Ainsi :
* $f(z) = f(q,d) = c^T r - v^T h(q,d)$ est la fonction que nous souhaitons minimiser
* Il y a deux variables de décision : $q$ et $r$.
* f est minimisée sous les contraintes $r >= Aq$ et $q > 0$

## Partie 2

5)
Pour résoudre ce problème, on peut utiliser un algorithme d'optimisation sous contraintes. Notre fonction est différentiable, mais n'est pas convexe, nous avons donc fait appel à un algorithme de descente de la librairie scipy.optimize.minimize: SLSQP (Sequential Least Squares Programming).

Si par exemple la fonction est convexe, il y a existence d'un point selle du Lagrangien, on peut donc utiliser l'algorithme d'Uzawa vu en cours.


In [19]:
import numpy as np
from scipy import optimize
from casadi import *
import time

alpha = 0.1

c = 0.001*np.array([30,1,1.3,4,1])
v = np.array([0.9,1.5,1.1])
d = np.array([400,67,33])
A = np.array([[3.5,2,1],
              [250,80,25],
              [0,8,3],
              [0,40,10],
              [0,8.5,0]])

def achat (q,A): 
    r = numpy.dot(q,np.transpose(A))
    return r

def h(q,d,alpha):
    h = np.zeros(len(d))
    for i in range(len(d)):
        h[i] = (q[i]*np.exp(-alpha*q[i])+d[i]*np.exp(-alpha*d[i]))/(np.exp(-alpha*q[i])+np.exp(-alpha*d[i]))
    return h

def profit (c,v,d,A,alpha): # profit théorique calculé pour q=d et r = Aq
    return np.dot(v,np.transpose(h(d,d,alpha))) - numpy.dot(c,np.transpose(achat(d,A)))

def f(qr): # qr est la concaténation de q et r, dans cet ordre. Nécessaire pour scipy.optimize qui prend en entré des vecteurs de dimension (n,1)
    q = qr[:3]
    r = qr[-5:]
    return numpy.dot(c,r) - np.dot(v,np.transpose(h(q,d,alpha)))

def contrainte1(qr): # On veut r > Aq
    q = qr[:3]
    r = qr[-5:]
    return  r - np.dot(A,q)

def contrainte2(qr):# On veut r>0 et q>0
    return qr
    

print("Résultat de l'agorithme : " + str(-optimize.minimize(f,np.zeros(8),method='SLSQP', constraints = [{'type':'ineq', 'fun':contrainte1},{'type':'ineq', 'fun':contrainte2}]).fun))
print("Profit théorique maximal pour q=d et r = Aq : "+ str(profit(c,v,d,A,alpha)))
print("Le résultat est proche du max théorique avec alpha=0.1.")

Résultat de l'agorithme : 326.34351219337486
Profit théorique maximal pour q=d et r = Aq : 330.17
Le résultat est proche du max théorique avec alpha=0.1.


6)
Les résultats obtenus montrent que les maxima sont atteints pour r = Aq et q=d, ce qui semble cohérent : inutile d'acheter plus que ce dont on a besoin pour produire, et inutile de produire plus que ce que les gens achèteront ; réciproquement, produire moins que la demande serait sous-optimal : il reste un profit à aller chercher (le coût marginal de production et le prix de vente étant constants, s'il est profitable de vendre à 10 personnes, il est aussi profitable de vendre à 20).

Pour alpha plus grand que 1, le profit max tend vers 0, car l'approximation de la fonction min par $h$ n'est pas rigoureuse  

7)
On cherche désormais à maximiser le profit espéré de notre problème, c'est à dire la fonction :
$$
PE(q,r) = \sum_{k} P(d^k,q,r)\pi^k
$$
où $P(d^k,q,r) = v^Th(q,r,d^k) - c^Tr$

On va donc minimiser $f(z) = -PE(z)$ où $z=q,r$

In [20]:
d1 = np.array([400,67,33])
d2 = np.array([500,80,53])
d3 = np.array([300,60,43])
p1 = 0.5
p2 = 0.3
p3 = 0.2

def f2(qr): # On minimise l'espérance du profit
    q = qr[:3]
    r = qr[-5:]
    return ( p1*(numpy.dot(c,r) - np.dot(v,np.transpose(h(q,d1,alpha)))) 
            +p2*(numpy.dot(c,r) - np.dot(v,np.transpose(h(q,d2,alpha))))
            +p3*(numpy.dot(c,r) - np.dot(v,np.transpose(h(q,d3,alpha)))))

print("Résultat de l'algorithme : " + str(-optimize.minimize(f2,np.zeros(8),method='SLSQP', constraints = [{'type':'ineq', 'fun':contrainte1},{'type':'ineq', 'fun':contrainte2}]).fun))

Résultat de l'algorithme : 316.65518699610425


Commentaire: 